### 1. Load the required libraries 

In [ ]:
from __future__ import print_function

import time
from PIL import Image
import numpy as np

from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave

### 2. Load Style and Content Images

In [ ]:
# fix the dimensions of the style and content images
height = 300
width = 300

content_image = Image.open('images/SAR.jpg')
content_image = content_image.resize((width, height))
content_image

In [ ]:
style_image = Image.open('images/andy.jpg')
style_image = style_image.resize((width, height))
style_image

### 3. Preprocess the Images

In [ ]:
# We need to convert the images into arrays for numerical processing
# The images occur in RGB pixel channel format creating the 3 dimensions in addition to length and width
# length x width x 3 dimensions
# We add an additional dimesnion of 1 to allow us to concatenate the style and content images

content_array = np.asarray(content_image, dtype='float32')
content_array = np.expand_dims(content_array, axis=0)
print(content_array.shape)

style_array = np.asarray(style_image, dtype='float32')
style_array = np.expand_dims(style_array, axis=0)
print(style_array.shape)


In [ ]:
# Subtract the mean RGB pixel value of the training set (ImageNet)
# invert the arrays from RGB to BGR

content_array[:, :, :, 0] -= 103.939
content_array[:, :, :, 1] -= 116.779
content_array[:, :, :, 2] -= 123.68
content_array = content_array[:, :, :, ::-1]

style_array[:, :, :, 0] -= 103.939
style_array[:, :, :, 1] -= 116.779
style_array[:, :, :, 2] -= 123.68
style_array = style_array[:, :, :, ::-1]


In [ ]:
# Create Keras' variables for the images

content_image = backend.variable(content_array)
style_image = backend.variable(style_array)
combination_image = backend.placeholder((1, height, width, 3))

# Concatenate the content and style images

input_tensor = backend.concatenate([content_image, style_image, combination_image], axis=0)


### 4. Load a Pre-Trained VGG-16 Model

In [ ]:

model = VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)


In [ ]:
# keep all layers except classification layers

layers = dict([(layer.name, layer.output) for layer in model.layers])
layers

### 5. Loss Functions

In [ ]:
# the loss function we want to minimise can be decomposed into three distinct parts: 
# 1) content loss 
# 2) style loss 
# 3) total variation loss.

# relative importance of these terms are determined by a set of scalar weights

content_weight = 0.1
style_weight = 1.0
total_variation_weight = 1.0

# initialize the total loss function and add to it

loss = backend.variable(0.)

###          a) Content Loss

In [ ]:
def content_loss(content, generated):
    return 0.5*(backend.sum(backend.square(content - generated)))

layer_features = layers['block2_conv1']
content_image_features = layer_features[0, :, :, :]
generated_image_features = layer_features[2, :, :, :]

loss += content_weight * content_loss(content_image_features, generated_image_features)

### b) Style Loss

In [ ]:
def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram

In [ ]:
def style_loss(style, generated):
    S = gram_matrix(style)
    G = gram_matrix(generated)
    channels = 3
    size = height * width
    return backend.sum(backend.square(S - G)) / (4. * (channels ** 2) * (size ** 2))

# ‘conv1 1’, ‘conv2 1’, ‘conv3 1’, ‘conv4 1’ and ‘conv5 1’

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

for layer_name in feature_layers:
    layer_features = layers[layer_name]
    style_image_features = layer_features[1, :, :, :]
    generated_image_features = layer_features[2, :, :, :]
    sl = style_loss(style_image_features, generated_image_features)
    loss += (style_weight / len(feature_layers)) * sl

### c) Total Variation Loss

In [ ]:
def total_variation_loss(x):
    a = backend.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
    b = backend.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
    return backend.sum(backend.pow(a + b, 1.25))

loss += total_variation_weight * total_variation_loss(combination_image)

### 6. Gradient Function and Loss Optimization 
Now that we have our input images massaged and our loss function calculators in place, all we have left to do is define gradients of the total loss relative to the combination image, and use these gradients to iteratively improve upon our combination image to minimise the loss.

In [ ]:
grads = backend.gradients(loss, combination_image)

outputs = [loss]
outputs += grads
f_outputs = backend.function([combination_image], outputs)

def eval_loss_and_grads(x):
    x = x.reshape((1, height, width, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [ ]:
x = np.random.uniform(0, 255, (1, height, width, 3)) - 128.

iterations = 12

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

### 7. Image Construction

In [ ]:
x = x.reshape((height, width, 3))
x = x[:, :, ::-1]
x[:, :, 0] += 103.939
x[:, :, 1] += 116.779
x[:, :, 2] += 123.68
x = np.clip(x, 0, 255).astype('uint8')

Image.fromarray(x).save('images/experiments 2/exp.jpg')

### There are several resources available for implementing Neural Style Tranfer (NST) using Tensorflow and Keras in Python. hnarayanan from Github offers a clear and concise implementation of NST and I have used his work as a framework for my own.

link to hnarayanan's work: https://github.com/hnarayanan/artistic-style-transfer/blob/master/notebooks/6_Artistic_style_transfer_with_a_repurposed_VGG_Net_16.ipynb